In [621]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
from importlib import reload
import pandas as pd
import numpy as np
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
import re
import networkx as nx
from scipy import stats
import plotly
import kaleido

import nltk

import matplotlib.pyplot as plt
import seaborn as sns

# gensim parts
from gensim import corpora
from gensim import models

### scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

import plotly.graph_objects as go

import sddk

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library

In [622]:
import textnet

In [623]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [624]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [625]:
# to access gsheet, you need Google Service Account key json file
# I have mine located in my personal space on sciencedata.dk, so I read it from there:

# (1) read the file and parse its content
file_data = conf[0].get("https://sciencedata.dk/files/ServiceAccountsKey.json").json()
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

ECCE_AGT_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KPpPaeX215HR_fVrakvJp8aB6oZDhHFTcBw0MKLw6as/edit?usp=sharing")

In [664]:
AGT = sddk.read_file("SDAM_data/AGT/AGT_lemmatized_20201205.json", "df", conf)
AGT.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,sentences,n_sentences,lemmatized_sentences
0,tlg0001.tlg001.perseus-grc2.xml,Apollonius Rhodius,Argonautica,38822,tlg0001,tlg0001.tlg001,3 B.C.,-2.5,{'-2.5': 1},-2.5,pagan,Epici/-ae,"ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν μ...","[ἀρχόμενος σέο, Φοῖβε, παλαιγενέων κλέα φωτῶν ...",3252,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,tlg0003.tlg001.perseus-grc2.xml,Thucydides,The Peloponnesian War,150118,tlg0003,tlg0003.tlg001,5 B.C.,-4.5,{'-4.5': 1},-4.5,pagan,Historici/-ae,Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,[Θουκυδίδης Ἀθηναῖος ξυνέγραψε τὸν πόλεμον τῶν...,6068,"[[θουκυδίδης, Ἀθηναῖος, συγγράφω, πόλεμος, Πελ..."
2,tlg0004.tlg001.perseus-grc1.xml,Diogenes Laertius,Lives of Eminent Philosophers,110763,tlg0004,tlg0004.tlg001,A.D. 3,2.5,{'2.5': 1},NaN,,Biographi,Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,[Τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβά...,10245,"[[φιλοσοφία, ἔργον, ἔνιοι, φημί, βάρβαρος, ἄρχ..."
3,tlg0005.tlg001.perseus-grc1.xml,Theocritus,Idylls,19200,tlg0005,tlg0005.tlg001,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,"̔Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα,...","[Αδύ τι τὸ ψιθύρισμα καὶ ἁ πίτυς αἰπόλε τήνα, ...",1982,"[[ψιθύρισμα, πίτυς, αἰπόλος, τῆνος, πηγή, μελί..."
4,tlg0005.tlg002.perseus-grc1.xml,Theocritus,Epigrams,1734,tlg0005,tlg0005.tlg002,4-3 B.C.,-3.0,"{'-3.5': 0.5, '-2.5': 0.5}",NaN,,Bucolici,τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ἕ...,[τὰ ῥόδα τὰ δροσόεντα καὶ ἁ κατάπυκνος ἐκείνα ...,152,"[[ῥόδον, δροσόεντα, κατάπυκνος, ἐκεῖνος, ἕρπυλ..."


In [665]:
def flat_lemmata(sentences):
    lemmata = [word for sent in sentences for word in sent]
    return lemmata
AGT["lemmata"] = AGT["lemmatized_sentences"].apply(flat_lemmata)

AGT["lemmata_wordcount"] = AGT.apply(lambda row: len(row["lemmata"]), axis=1)
AGT["lemmata_wordcount"].sum() # previously we had 13925726, then 13713183, finally 14756899

14373580

In [666]:
def get_flat_sentences(series):
    sentences_list = [sent for doc in series.tolist() for sent in doc]
    return sentences_list

In [667]:
subcorpora_dict = {
    "Pagan (8th c. BCE - 4th c. CE)" : AGT[(AGT["provenience"]=="pagan") & (AGT["date_avr"] < 4)],
    "Pagan (1st - 4th c. CE)" : AGT[(AGT["date_avr"].between(0, 4)) & (AGT["provenience"]=="pagan")],
    "Septuagint" : AGT[AGT["author_id"]=="tlg0527"],
    "Philo of Alexandria" : AGT[AGT["author_id"]=="tlg0018"],
    "Titus Flavius Josephus" : AGT[AGT["author_id"]=="tlg0526"],
    "Paul of Tarsus" : AGT[AGT["author_id"]=="tlg0031paul"],
    "New Testament" : AGT[AGT["author_id"].str.startswith("tlg0031")],
    "Christian (1st - 4th c. CE)" : AGT[(AGT["date_avr"] < 4) & (AGT["provenience"]=="christian")]}

In [668]:
subselections = []
for key in subcorpora_dict.keys():
    subselection = subcorpora_dict[key]
    subselection_dict = {}
    wordcount = subselection["wordcount"].sum()
    doccount = len(subselection)
    sents = get_flat_sentences(subselection["lemmatized_sentences"])
    sents_len = len(sents)
    subselection_dict.update({"label" : key, "wordcount" : wordcount, "doccount" : doccount, "n_sentences": sents_len,"sentences" : sents})
    subselections.append(subselection_dict)
subselections_df = pd.DataFrame(subselections)
subselections_df

,label,wordcount,doccount,n_sentences,sentences
0,Pagan (8th c. BCE - 4th c. CE),12965219,774,1039409,"[[ἄρχω, Φοῖβος, παλαιγενής, κλέος, φώς, μιμνήσ..."
1,Pagan (1st - 4th c. CE),8692448,411,723956,"[[γεωγραφία, σόσσιε, σενεκίων, ἱστορικός, διαφ..."
2,Septuagint,799083,55,57856,"[[ποιέω, θεός, οὐρανός, γῆ], [γῆ, εἰμί, ἀόρατο..."
3,Philo of Alexandria,643011,31,82966,"[[ἄλλος, νομοθέτης, ἀκαλλώπιστος, γυμνός, νομι..."
4,Titus Flavius Josephus,469580,4,19707,"[[ἱστορία, συγγράφω, βούλομαι, αὐτός, ὁράω, σπ..."
5,Paul of Tarsus,24066,7,1632,"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό..."
6,New Testament,137787,27,8698,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
7,Christian (1st - 4th c. CE),2975644,131,291084,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."


# term2term & term2doc matraces with sklearn - Paul as example

In [687]:
# let's explore Pa
docs = subselections_df.at[5, "sentences"]

In [688]:
len(docs)

1632

In [689]:
AGT_C = subcorpora_dict["Christian (1st - 4th c. CE)"]
len(AGT_C)

131

In [690]:
AGT_C.head(5)

,filename,author,title,wordcount,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,tlg_epithet,clean_string,sentences,n_sentences,lemmatized_sentences,lemmata,lemmata_wordcount
385,tlg0031.tlg001.perseus-grc2.xml,None,New Testament - Matthew,18288,tlg0031matt,tlg0031.tlg001,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱο...,[ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱο...,1276,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυίδ...",8551
386,tlg0031.tlg002.perseus-grc2.xml,None,New Testament - Mark,11274,tlg0031mark,tlg0031.tlg002,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΑΡΧΗ τοῦ εὐαγγελίου Ἰησοῦ Χριστοῦ . Καθὼς γέγ...,"[ΑΡΧΗ τοῦ εὐαγγελίου Ἰησοῦ Χριστοῦ ., Καθὼς γέ...",790,"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ...","[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσα...",5243
387,tlg0031.tlg003.perseus-grc2.xml,Luke (the evangelist),New Testament - Luke,19458,tlg0031luke,tlg0031.tlg003,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχείρησαν ἀνατάξασθαι διήγ...,[ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχείρησαν ἀνατάξασθαι διήγ...,1274,"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πληρ...",9107
388,tlg0031.tlg004.perseus-grc2.xml,Johnannine literature,New Testament - John,15590,tlg0031john,tlg0031.tlg004,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],"ΕΝ ΑΡΧΗ ἦν ὁ λόγος, καὶ ὁ λόγος ἦν πρὸς τὸν θ...","[ΕΝ ΑΡΧΗ ἦν ὁ λόγος, καὶ ὁ λόγος ἦν πρὸς τὸν θ...",1164,"[[ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεός, ...","[ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεός, ε...",7001
389,tlg0031.tlg005.perseus-grc2.xml,Luke (the evangelist),New Testament - Acts,18406,tlg0031luke,tlg0031.tlg005,A.D. 1,0.5,{'0.5': 1},0.5,christian,[],"τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων, ...","[τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων, ...",960,"[[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰ...","[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰη...",9042


In [691]:
author = AGT_C["author"][10]
author

'Paul of Tarsus'

In [692]:
sub_df = AGT_C[AGT_C["author_id"]=="tlg0031john"]
sub_df["date_probs"]

388    {'0.5': 1}
407    {'0.5': 1}
408    {'0.5': 1}
409    {'0.5': 1}
Name: date_probs, dtype: object

In [693]:
get_flat_sentences(sub_df["lemmatized_sentences"])

[['ἀρχή', 'εἰμί', 'λόγος', 'λόγος', 'εἰμί', 'θεός', 'θεός', 'εἰμί', 'λόγος'],
 ['ἀρχή', 'θεός'],
 ['πᾶς', 'γίγνομαι', 'γίγνομαι', 'εἷς'],
 ['γίγνομαι', 'ζωή', 'εἰμί', 'ζωή', 'φῶς', 'ἄνθρωπος'],
 ['φῶς', 'σκοτία', 'φαίνω', 'σκοτία', 'καταλαμβάνω'],
 ['γίγνομαι', 'ἄνθρωπος', 'ἀποστέλλω', 'θεός', 'ὄνομα', 'ἰωάνης'],
 ['οὗτος', 'ἔρχομαι', 'μαρτυρία', 'μαρτυρέω', 'φῶς', 'πᾶς', 'πιστεύω'],
 ['εἰμί', 'ἐκεῖνος', 'φῶς', 'μαρτυρέω', 'φῶς'],
 ['φῶς', 'ἀληθινός', 'φωτίζω', 'ἄνθρωπος', 'ἔρχομαι', 'κόσμος'],
 ['κόσμος', 'εἰμί', 'κόσμος', 'γίγνομαι', 'κόσμος', 'γιγνώσκω'],
 ['ἴδιος', 'ἔρχομαι', 'ἴδιος', 'παραλαμβάνω'],
 ['λαμβάνω',
  'δίδωμι',
  'ἐξουσία',
  'τέκνον',
  'θεός',
  'γίγνομαι',
  'πιστεύω',
  'ὄνομα',
  'αἷμα',
  'θέλημα',
  'σάρξ',
  'θέλημα',
  'ἀνήρ',
  'θεός',
  'γεννάω'],
 ['λόγος',
  'σάρξ',
  'γίγνομαι',
  'σκηνόω',
  'θεάομαι',
  'δόξα',
  'δόξα',
  'μονογενής',
  'πατήρ',
  'πλήρης',
  'χάρις',
  'ἀλήθεια'],
 [],
 ['ἰωάνης',
  'μαρτυρέω',
  'κράζω',
  'λέγω',
  'οὗτος',
  'λέγω

In [694]:
probs_dict = {}
for d in sub_df["date_probs"]:
    probs_dict.update(d)
probs_dict

{'0.5': 1}

# Comparing All Christian Authors

In [695]:
def grouping_gnt(author_id):
    if author_id == "tlg0031a":
        author_id = "tlg0031matt"
    if author_id == "tlg0031b":
        author_id = "tlg0031mark"
    if re.match("tlg0031\w?$", author_id):
        return "tlg0031rest"
    else:
        return author_id
new_author_ids = AGT_C["author_id"].apply(grouping_gnt)

AGT_C["author_id"] = new_author_ids

In [696]:
authors_dicts = []
for author_id in AGT_C["author_id"].unique():
    sub_df = AGT_C[AGT_C["author_id"]==author_id]
    authors_dict = {}
    authors_dict["author_id"] = author_id
    authors_dict["author"] = sub_df["author"][0]
    authors_dict["n_titles"] = len(sub_df)
    authors_dict["titles"] = sub_df["title"].tolist()
    authors_dict["filenames"] = sub_df["filename"].tolist()
    authors_dict["wordcount"] = sub_df["wordcount"].sum()
    authors_dict["lemmata_wordcount"] = sub_df["lemmata_wordcount"].sum()
    authors_dict["n_sentences"] = sub_df["n_sentences"].sum()
    probs_dict = {}
    for d in sub_df["date_probs"]:
        probs_dict.update(d)
    authors_dict["date_probs"] = probs_dict
    authors_dict["lemmata_lists"] = sub_df["lemmata"].tolist()
    authors_dict["lemmatized_sentences"] = get_flat_sentences(sub_df["lemmatized_sentences"])
    authors_dicts.append(authors_dict)
    
authors_df = pd.DataFrame(authors_dicts)
authors_df.head(5)

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,lemmatized_sentences
0,tlg0031matt,None,1,[New Testament - Matthew],[tlg0031.tlg001.perseus-grc2.xml],18288,8551,1276,{'0.5': 1},"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
1,tlg0031mark,None,1,[New Testament - Mark],[tlg0031.tlg002.perseus-grc2.xml],11274,5243,790,{'0.5': 1},"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσ...","[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ..."
2,tlg0031luke,Luke (the evangelist),2,"[New Testament - Luke, New Testament - Acts]","[tlg0031.tlg003.perseus-grc2.xml, tlg0031.tlg0...",37864,18149,2234,{'0.5': 1},"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη..."
3,tlg0031john,Johnannine literature,4,"[New Testament - John, New Testament - 1 John,...","[tlg0031.tlg004.perseus-grc2.xml, tlg0031.tlg0...",18189,8151,1339,{'0.5': 1},"[[ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεός, ...","[[ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεός, ..."
4,tlg0031paul,Paul of Tarsus,7,"[New Testament - Romans, New Testament - 1 Cor...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11077,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό..."


In [697]:
clean_titles = authors_df["titles"].apply(lambda cell: [title.rpartition(" - ")[2] for title in cell]) 
authors_df["titles"] = clean_titles

In [698]:
def fill_author(author, titles):
    if author == None:
        return titles[0]
    else:
        return author.partition(",")[0]
 
clean_authors = authors_df.apply(lambda row: fill_author(row["author"], row["titles"]), axis=1)
authors_df["author"] = clean_authors

In [699]:
authors_df.loc[authors_df["author_id"]=="tlg0031rest", "author"] = "GNT rest" #, ["author"]] = 


In [700]:
authors_df.loc[authors_df["author_id"]=="tlg1484", "author"] = "Martyrium Polycarpi"
authors_df.loc[authors_df["author_id"]=="tlg1311", "author"] = "Didache" #, ["author"]] = #, ["author"]] = 

In [701]:
authors_df

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,lemmatized_sentences
0,tlg0031matt,Matthew,1,[Matthew],[tlg0031.tlg001.perseus-grc2.xml],18288,8551,1276,{'0.5': 1},"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...","[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί..."
1,tlg0031mark,Mark,1,[Mark],[tlg0031.tlg002.perseus-grc2.xml],11274,5243,790,{'0.5': 1},"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός, γράφω, ἠσ...","[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ..."
2,tlg0031luke,Luke (the evangelist),2,"[Luke, Acts]","[tlg0031.tlg003.perseus-grc2.xml, tlg0031.tlg0...",37864,18149,2234,{'0.5': 1},"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...","[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη..."
3,tlg0031john,Johnannine literature,4,"[John, 1 John, 2 John, 3 John]","[tlg0031.tlg004.perseus-grc2.xml, tlg0031.tlg0...",18189,8151,1339,{'0.5': 1},"[[ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεός, ...","[[ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεός, ..."
4,tlg0031paul,Paul of Tarsus,7,"[Romans, 1 Corinthians, 2 Corinthians, Galatia...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11077,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό..."
5,tlg0031pspa,Pauline literature,6,"[Ephesians, Colossians, 2 Thessalonians, 1 Tim...","[tlg0031.tlg010.perseus-grc2.xml, tlg0031.tlg0...",8324,4102,390,{'0.5': 1},"[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...","[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ..."
6,tlg0031rest,GNT rest,6,"[Hebrews, James, 1 Peter, 2 Peter, Jude, Revel...","[tlg0031.tlg019.perseus-grc2.xml, tlg0031.tlg0...",19782,9731,1037,{'0.5': 1},"[[πολύτροπος, θεός, λαλέω, πατήρ, προφήτης, ἔσ...","[[πολύτροπος, θεός, λαλέω, πατήρ, προφήτης, ἔσ..."
7,tlg0555,Clement of Alexandria,7,"[Protrepticus, Paedagogus, Stromata (Books I-V...","[tlg0555.tlg001.perseus-grc1.xml, tlg0555.tlg0...",235754,52918,14753,"{'1.5': 0.5, '2.5': 0.5}","[[ἀμφίων, Θηβαῖος, Ἄριοι, μηθυμναῖος, ἄμφω, εἰ...","[[ἀμφίων, Θηβαῖος, Ἄριοι, μηθυμναῖος, ἄμφω, εἰ..."
8,tlg0563,Maximus of Tyre,1,[Dialexeis],[tlg0563.tlg001.1st1K-grc1.xml],118960,50873,22598,{'1.5': 1},"[[ὑπόθεσις, ἁρμόσεταιμαξίμου, φιλοσοφοὺμενα, φ...","[[ὑπόθεσις, ἁρμόσεταιμαξίμου, φιλοσοφοὺμενα, φ..."
9,tlg0645,Justinus Martyr,3,"[Apologia, Apology II, Dialogus cum Tryphone]","[tlg0645.tlg001.opp-grc1.xml, tlg0645.tlg002.p...",73449,33795,5829,{'1.5': 1},"[[αὐτοκράτωρ, Τίτος, αἰλίῳ, ἀδριανῷ, ἀντωνίνῳ,...","[[αὐτοκράτωρ, Τίτος, αἰλίῳ, ἀδριανῷ, ἀντωνίνῳ,..."


In [702]:
# COUNT δικαιοσύνη IN SENTENCES
def count_term_in_sents(lemmatized_sentences):
    return sum([sent.count("δικαιοσύνη") for sent in lemmatized_sentences])

authors_df["dikaiosyne_N"]  = authors_df["lemmatized_sentences"].apply(count_term_in_sents)

In [703]:
authors_df["dikaiosyne_TF"]  = authors_df.apply(lambda row: row["dikaiosyne_N"] / row["lemmata_wordcount"], axis=1)
authors_df.sort_values("dikaiosyne_TF", ascending=False, inplace=True)

In [704]:
authors_df

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,lemmatized_sentences,dikaiosyne_N,dikaiosyne_TF
20,tlg1622,Polycarp,1,[Epistula ad Philippenses],[tlg1622.tlg001.1st1K-grc1.xml],1804,965,212,"{'0.5': 0.5, '1.5': 0.5}","[[πολύκαρπος, πρέσβυς, ἐκκλησία, θεός, παροικο...","[[πολύκαρπος, πρέσβυς, ἐκκλησία, θεός, παροικο...",8,0.008290
4,tlg0031paul,Paul of Tarsus,7,"[Romans, 1 Corinthians, 2 Corinthians, Galatia...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11077,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...","[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...",49,0.004424
13,tlg1271,Clemens Romanus,2,"[Epistula I ad Corinthios, Epistula II ad Cori...","[tlg1271.tlg001.1st1K-grc1.xml, tlg1271.tlg002...",13936,6298,1410,{'0.5': 1},"[[ἐκκλησία, θεός, παροικοῦσα, Ῥώμη, ἐκκλησία, ...","[[ἐκκλησία, θεός, παροικοῦσα, Ῥώμη, ἐκκλησία, ...",21,0.003334
10,tlg0646,Pseudo-Justinus Martyr,1,[Epistula ad Diognetum],[tlg0646.tlg004.1st1K-grc1.xml],2725,1353,240,"{'2.5': 0.5, '4.5': 0.5}","[[ὁράω, κράτιστος, διόγνητε, ὑπερεσπουδακότα, ...","[[ὁράω, κράτιστος, διόγνητε, ὑπερεσπουδακότα, ...",4,0.002956
12,tlg1216,Barnabas,1,[The Epistle of Barnabas],[tlg1216.tlg001.perseus-grc2.xml],7440,3465,875,"{'0.5': 0.5, '1.5': 0.5}","[[χαίρω, υἱός, θυγάτηρ, ὄνομα, κύριος, ἀγαπάω,...","[[χαίρω, υἱός, θυγάτηρ, ὄνομα, κύριος, ἀγαπάω,...",9,0.002597
16,tlg1419,Hermas,1,[The Shepherd of Hermas],[tlg1419.tlg001.1st1K-grc1.xml],30282,14279,2959,{'1.5': 1},"[[τρέφω, πέρνημι, ῥόδῃ, τὶς, Ῥώμη, πολύς, ἔτος...","[[τρέφω, πέρνημι, ῥόδῃ, τὶς, Ῥώμη], [πολύς, ἔτ...",33,0.002311
5,tlg0031pspa,Pauline literature,6,"[Ephesians, Colossians, 2 Thessalonians, 1 Tim...","[tlg0031.tlg010.perseus-grc2.xml, tlg0031.tlg0...",8324,4102,390,{'0.5': 1},"[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...","[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...",8,0.001950
14,tlg1311,Didache,1,[Didache XII Apostolorum],[tlg1311.tlg001.1st1K-grc1.xml],2377,1053,284,{'1.5': 1},"[[ὁδός, δύο, εἰμί, εἷς, ζωή, εἷς, θάνατος, δια...","[[ὁδός, δύο, εἰμί, εἷς, ζωή, εἷς, θάνατος, δια...",2,0.001899
6,tlg0031rest,GNT rest,6,"[Hebrews, James, 1 Peter, 2 Peter, Jude, Revel...","[tlg0031.tlg019.perseus-grc2.xml, tlg0031.tlg0...",19782,9731,1037,{'0.5': 1},"[[πολύτροπος, θεός, λαλέω, πατήρ, προφήτης, ἔσ...","[[πολύτροπος, θεός, λαλέω, πατήρ, προφήτης, ἔσ...",17,0.001747
9,tlg0645,Justinus Martyr,3,"[Apologia, Apology II, Dialogus cum Tryphone]","[tlg0645.tlg001.opp-grc1.xml, tlg0645.tlg002.p...",73449,33795,5829,{'1.5': 1},"[[αὐτοκράτωρ, Τίτος, αἰλίῳ, ἀδριανῷ, ἀντωνίνῳ,...","[[αὐτοκράτωρ, Τίτος, αἰλίῳ, ἀδριανῷ, ἀντωνίνῳ,...",45,0.001332


# Networks for authors

In [705]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [706]:
nx_from_sentences = authors_df["lemmatized_sentences"].apply(textnet.network_from_sentences)
nx_from_lemmata = authors_df["lemmata_lists"].apply(textnet.network_from_lemmata_lists)

In [707]:
authors_df["G_sents"] = nx_from_sentences
authors_df["G_lems"] = nx_from_lemmata

In [708]:
def ego_network(G):
    try:
        G_ego = textnet.construct_ego_network(G, "δικαιοσύνη", 30, reduced=False)
    except:
        G_ego = nx.Graph()
        G_ego.clear()
    return G_ego

In [709]:
authors_df["dikaiosyne_G_sents"] = authors_df["G_sents"].apply(ego_network)
authors_df["dikaiosyne_G_lems"] = authors_df["G_lems"].apply(ego_network)

In [710]:
list(authors_df.iloc[0]["dikaiosyne_G_sents"].nodes())

['μωμοσκοπεῖται',
 'φιλαργυρία',
 'ἐντυγχανούσας',
 'ἐντολή',
 'πρέσβυς',
 'θυσιαστήριον',
 'πᾶς',
 'εὔσπλαγχνος',
 'χαλιναγωγοῦντες',
 'οὗτος',
 'γινωσκούσας',
 'ὀφειλέτης',
 'θεός',
 'ἁγνεία',
 'ὀργή',
 'λανθάνω',
 'ἄμεμπτος',
 'διαβολή',
 'δικαιοσύνη',
 'καταλαλιά',
 'προσωποληψίας',
 'κύριος',
 'εἰμί',
 'πίστις',
 'οὐδείς',
 'κακός',
 'προνοοῦντες',
 'κρίσις',
 'ψευδομαρτυρία',
 'ἀπεχόμενοι',
 'ἁμαρτία']

# Terms within sentences

In [711]:
def sentences_with_term(sentences, term, flatlist=False):
    sentences_with_term = [sen for sen in sentences if term in sen]
    if flatlist == True:
        flatlist = [word for sen in sentences_with_term for word in sen]
        flatlist = [word for word in flatlist if word != "δικαιοσύνη"]
        return flatlist
    else: 
        return sentences_with_term
authors_df["dik_sents_flat"] = authors_df["lemmatized_sentences"].apply(lambda cell: sentences_with_term(cell, "δικαιοσύνη", flatlist=True))

In [712]:
def get_tf(lemmata):
    fd_list = list(nltk.FreqDist(lemmata).most_common())
    tf_list = [(tup[0], np.round(tup[1] / len(lemmata), 5)) for tup in fd_list]
    return tf_list

authors_df["dik_sents_TF"] = authors_df["dik_sents_flat"].apply(get_tf)

In [713]:
def term_idf(term, all_sentences):
    idf_raw =  len(all_sentences) / len([sent for sent in all_sentences if term in sent])
    idf_log = math.log(idf_raw)
    return idf_log

def get_tfidf(key_term_lemmata, all_sentences):
    fd_list = list(nltk.FreqDist(key_term_lemmata).most_common())
    tfidf_list = [(tup[0], np.round((tup[1] / len(key_term_lemmata) * term_idf(tup[0], all_sentences)), 5)) for tup in fd_list]
    tfidf_list = sorted(tfidf_list, key=lambda tup: tup[1], reverse=True)
    return tfidf_list

authors_df["dik_sents_TFIDF"] = authors_df.apply(lambda x: get_tfidf(x["dik_sents_flat"], x["lemmatized_sentences"]), axis=1)

In [714]:
authors_df["dik_sents_TFIDF_10"] = authors_df["dik_sents_TFIDF"].apply(lambda cell: [tup[0] for tup in cell[:10]])

In [715]:
authors_df.head(5)

,author_id,author,n_titles,titles,filenames,wordcount,lemmata_wordcount,n_sentences,date_probs,lemmata_lists,...,dikaiosyne_N,dikaiosyne_TF,G_sents,G_lems,dikaiosyne_G_sents,dikaiosyne_G_lems,dik_sents_flat,dik_sents_TF,dik_sents_TFIDF,dik_sents_TFIDF_10
20,tlg1622,Polycarp,1,[Epistula ad Philippenses],[tlg1622.tlg001.1st1K-grc1.xml],1804,965,212,"{'0.5': 0.5, '1.5': 0.5}","[[πολύκαρπος, πρέσβυς, ἐκκλησία, θεός, παροικο...",...,8,0.008290,"(ξρεσξεντεμ, περιέχω, τολεραντια, ἀγαπάω, μήτη...","(Χριστός, Ἰησοῦς, εἰμί, πᾶς, κύριος, νεκρός, ἐ...","(μωμοσκοπεῖται, φιλαργυρία, ἐντυγχανούσας, ἐντ...","(εἰμί, πᾶς, ετ, ϝος, εἶμι, ϝοβις, αβ, αϝαριτια...","[μακάριος, πτωχός, διώκω, εἰμί, βασιλεία, θεάο...","[(εἰμί, 0.04651), (οὗτος, 0.04651), (ἐντολή, 0...","[(οὗτος, 0.18466), (ἐντολή, 0.18466), (πτωχός,...","[οὗτος, ἐντολή, πτωχός, διώκω, θεάομαι, ἀδελφό..."
4,tlg0031paul,Paul of Tarsus,7,"[Romans, 1 Corinthians, 2 Corinthians, Galatia...","[tlg0031.tlg006.perseus-grc2.xml, tlg0031.tlg0...",24066,11077,1632,{'0.5': 1},"[[Παῦλος, δοῦλος, Ἰησοῦς, Χριστός, κλητός, ἀπό...",...,49,0.004424,"(ἄτακτος, βεβαιόω, τυφλός, καθοράω, κληρονομία...","(Χριστός, Ἰησοῦς, κύριος, θεός, χάρις, πατήρ, ...","(χριστός, ἔχω, δίδωμι, κύριος, ἔθνος, εἰμί, πᾶ...","(Χριστός, Ἰησοῦς, κύριος, θεός, χάρις, πατήρ, ...","[θεός, ἀποκαλύπτω, πίστις, πίστις, γράφω, δίκα...","[(θεός, 0.04651), (πίστις, 0.04134), (νόμος, 0...","[(πίστις, 0.12679), (ἁμαρτία, 0.11135), (νόμος...","[πίστις, ἁμαρτία, νόμος, λογίζομαι, θεός, βασι..."
13,tlg1271,Clemens Romanus,2,"[Epistula I ad Corinthios, Epistula II ad Cori...","[tlg1271.tlg001.1st1K-grc1.xml, tlg1271.tlg002...",13936,6298,1410,{'0.5': 1},"[[ἐκκλησία, θεός, παροικοῦσα, Ῥώμη, ἐκκλησία, ...",...,21,0.003334,"(ἐμπορία, περιέχω, ἔξοχος, βεβαιόω, χρυσός, ἀπ...","(Χριστός, Ἰησοῦς, κύριος, λέγω, πνεῦμα, ἅγιος,...","(χριστός, ἔχω, ποιέω, λαμβάνω, δίδωμι, κύριος,...","(Χριστός, Ἰησοῦς, κύριος, λέγω, πνεῦμα, ἅγιος,...","[οὗτος, ἄπειμι, εἰρήνη, ἀπολιμπάνω, ἕκαστος, φ...","[(θεός, 0.0283), (πίστις, 0.02358), (γίγνομαι,...","[(καυχάομαι, 0.10261), (πίστις, 0.10037), (πύλ...","[καυχάομαι, πίστις, πύλη, θεός, δύσις, ἀγάπη, ..."
10,tlg0646,Pseudo-Justinus Martyr,1,[Epistula ad Diognetum],[tlg0646.tlg004.1st1K-grc1.xml],2725,1353,240,"{'2.5': 0.5, '4.5': 0.5}","[[ὁράω, κράτιστος, διόγνητε, ὑπερεσπουδακότα, ...",...,4,0.002956,"(καταγέλαστος, προσδοκάω, τρυφή, πεφανέρωτο, ἄ...","(Χριστιανός, κόσμος, γνῶσις, ξύλον, θεός, οὗτο...","(πολύς, δεῖγμα, πρόσκαιρος, κρυβῇ, ὑπομένω, γῆ...",(δικαιοσύνη),"[ἐφηδόμενος, ἁμάρτημα, ἀνεχόμενος, ἀδικία, και...","[(ἐκεῖνος, 0.05405), (πολύς, 0.05405), (πῦρ, 0...","[(ἐκεῖνος, 0.25878), (πῦρ, 0.20925), (πολύς, 0...","[ἐκεῖνος, πῦρ, πολύς, ἐφηδόμενος, ἁμάρτημα, ἀν..."
12,tlg1216,Barnabas,1,[The Epistle of Barnabas],[tlg1216.tlg001.perseus-grc2.xml],7440,3465,875,"{'0.5': 0.5, '1.5': 0.5}","[[χαίρω, υἱός, θυγάτηρ, ὄνομα, κύριος, ἀγαπάω,...",...,9,0.002597,"(οἰκοδομήσουσιν, περιέχω, ἁγιάσατε, ὑποπόδιον,...","(κύριος, λέγω, προφήτης, οὗτος, θεός, Μωϋσῆς, ...","(ἀπόλλυμι, ἀκούω, γῆ, πᾶς, οὗτος, πνεῦμα, ἀσεβ...","(κύριος, λέγω, προφήτης, οὗτος, θεός, Μωϋσῆς, ...","[πείθω, οὗτος, σύνοιδα, λαλέω, πολύς, ἐπίσταμα...","[(ὁδός, 0.0411), (κύριος, 0.0274), (θεός, 0.02...","[(ὁδός, 0.17298), (σκότος, 0.13651), (ἀγαθός, ...","[ὁδός, σκότος, ἀγαθός, σύνοιδα, ἐπίσταμαι, συν..."


In [716]:
tfidf_words = list(set([word for wordlist in authors_df["dik_sents_TFIDF_10"].tolist() for word in wordlist]))
tfidf_words = [word for word in tfidf_words if not re.search("\W", word)]
print(tfidf_words)

['σκότος', 'ἄνθρωπος', 'μακάριος', 'ἐπιθυμία', 'δοῦλος', 'ἀγαπάω', 'ἐπίσταμαι', 'τέλειος', 'προσγινομένας', 'φῶς', 'πλημμελέω', 'προεπεκαλέσασθέ', 'ὅμοιος', 'ἀγάπη', 'γεννάω', 'συνώδευσεν', 'βεβαιωτής', 'ἐφηδόμενος', 'ἐργάζομαι', 'ἀποχή', 'ἐκφέρω', 'εὐλογητός', 'ὑπάγω', 'πολύς', 'ἀναγκάζω', 'κύριος', 'εὐσέβεια', 'γένος', 'ἀδελφός', 'χείρων', 'ἐλευθερωτής', 'ὁδός', 'ὑποδιαιρούμεναι', 'μιμνῄσκομαι', 'προπορεύσεται', 'καυχάομαι', 'πρέπω', 'Χριστός', 'βασιλεία', 'τύραννος', 'Σαλήμ', 'ἄγγελος', 'μέρος', 'προστίθημι', 'ὑποστρέφω', 'ἀνομία', 'πεινῶντες', 'προφήτης', 'ὁμολογήσαντα', 'ἀνίστημι', 'διώκω', 'ἐπαινουμένας', 'δίκαιος', 'θεός', 'ψεῦδος', 'σωφροσύνη', 'ἀνάξιος', 'πατήρ', 'ἐπιτρέπω', 'ἐλέγχω', 'διορίζων', 'πλοῦτος', 'βασιλεύω', 'διψῶντες', 'στέφανος', 'οἷος', 'ἀνατελεῖ', 'ποιέω', 'ἐφώπλισεν', 'στόμα', 'κατορθωτὴς', 'πράσσω', 'κᾀκεῖνον', 'ψυχή', 'διῶκται', 'δέχομαι', 'ὑπερβολή', 'πονηρία', 'ἐκεῖνος', 'βασιλεύς', 'λόγος', 'ἴσος', 'γενικαῖς', 'ἐντολή', 'λέγω', 'θεάομαι', 'οὐτος', 'ὁρίζω',

In [717]:
set_with_dataframe(ECCE_AGT_overview.add_worksheet("tfidf_words_20201205", 1, 1), pd.DataFrame(tfidf_words))

In [661]:
authors_df.columns

Index(['author_id', 'author', 'n_titles', 'titles', 'filenames', 'wordcount',
       'lemmata_wordcount', 'n_sentences', 'date_probs', 'lemmata_lists',
       'lemmatized_sentences', 'dikaiosyne_N', 'dikaiosyne_TF', 'G_sents',
       'G_lems', 'dikaiosyne_G_sents', 'dikaiosyne_G_lems', 'dik_sents_flat',
       'dik_sents_TF', 'dik_sents_TFIDF', 'dik_sents_TFIDF_10'],
      dtype='object')

In [300]:
columns_to_omit = ["lemmatized_sentences", "lemmata_lists", "G_sents", "G_lems", "dikaiosyne_G_sents" , "dikaiosyne_G_lems"]

In [301]:
set_with_dataframe(ECCE_AGT_overview.add_worksheet("authors_df_20201202", 1,1), authors_df.drop(columns_to_omit, axis=1))

# N of shared terms within dik_sents_TFIDF_10

In [718]:
authors_df["dik_sents_TFIDF_10"].tolist()[:5]

[['οὗτος',
  'ἐντολή',
  'πτωχός',
  'διώκω',
  'θεάομαι',
  'ἀδελφός',
  'ἐπιτρέπω',
  'προεπεκαλέσασθέ',
  'πληρόω',
  'ἐκφέρω'],
 ['πίστις',
  'ἁμαρτία',
  'νόμος',
  'λογίζομαι',
  'θεός',
  'βασιλεύω',
  'δοῦλος',
  'παρίστημι',
  'πιστεύω',
  'Χριστός'],
 ['καυχάομαι',
  'πίστις',
  'πύλη',
  'θεός',
  'δύσις',
  'ἀγάπη',
  'πατήρ',
  'ὑπομονή',
  'ἄνθρωπος',
  'Ἀβραάμ'],
 ['ἐκεῖνος',
  'πῦρ',
  'πολύς',
  'ἐφηδόμενος',
  'ἁμάρτημα',
  'ἀνεχόμενος',
  'συνευδοκέω',
  'ἐλεγχθέντες',
  'ἀνάξιος',
  'καλύπτω'],
 ['ὁδός',
  'σκότος',
  'ἀγαθός',
  'σύνοιδα',
  'ἐπίσταμαι',
  'συνώδευσεν',
  'ἀναγκάζω',
  'ἀνατελεῖ',
  'προπορεύσεται',
  'περιστελεῖ']]

In [719]:
edges_tuples = []
for author1 in authors_df["author"].tolist():
    author1_words = authors_df.loc[authors_df["author"]==author1, "dik_sents_TFIDF_10"].values[0]
    for author2 in authors_df["author"].tolist():
        if author2 != author1:
            author2_words = authors_df.loc[authors_df["author"]==author2, "dik_sents_TFIDF_10"].values[0]
            shared = len(set(author1_words) & set(author2_words))
            if shared > 1:
                edges_tuples.append((author1, author2, shared))

In [596]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [750]:
G = nx.Graph()
G.clear()
G.add_weighted_edges_from(edges_tuples)
for (u, v, wt) in G.edges.data('weight'):
    G[u][v]["weight"] = int(wt)
total_weight = sum([int(n) for n in nx.get_edge_attributes(G, "weight").values()])
for (u, v) in G.edges:
    G[u][v]["norm_weight"] = round((G[u][v]["weight"] / total_weight), 5)
    G[u][v]["distance"] = round(1 / (G[u][v]["weight"]), 5)
    G[u][v]["norm_distance"] = round(1 / (G[u][v]["norm_weight"] ), 5)
G.remove_edges_from(nx.selfloop_edges(G))

In [751]:
G.edges.data()

EdgeDataView([('Paul of Tarsus', 'Clemens Romanus', {'weight': 2, 'norm_weight': 0.02222, 'distance': 0.5, 'norm_distance': 45.0045}), ('Paul of Tarsus', 'Origenes', {'weight': 2, 'norm_weight': 0.02222, 'distance': 0.5, 'norm_distance': 45.0045}), ('Paul of Tarsus', 'Theophilus Antiochenus', {'weight': 3, 'norm_weight': 0.03333, 'distance': 0.33333, 'norm_distance': 30.003}), ('Paul of Tarsus', 'Eusebius', {'weight': 2, 'norm_weight': 0.02222, 'distance': 0.5, 'norm_distance': 45.0045}), ('Paul of Tarsus', 'Epiphanius', {'weight': 5, 'norm_weight': 0.05556, 'distance': 0.2, 'norm_distance': 17.99856}), ('Clemens Romanus', 'Pauline literature', {'weight': 2, 'norm_weight': 0.02222, 'distance': 0.5, 'norm_distance': 45.0045}), ('Clemens Romanus', 'GNT rest', {'weight': 2, 'norm_weight': 0.02222, 'distance': 0.5, 'norm_distance': 45.0045}), ('Clemens Romanus', 'Clement of Alexandria', {'weight': 2, 'norm_weight': 0.02222, 'distance': 0.5, 'norm_distance': 45.0045}), ('Clemens Romanus', '

In [752]:
reload(textnet)

<module 'textnet' from '/home/kasev/ECCE_AGT/scripts/textnet.py'>

In [753]:
fig = textnet.draw_2d_network(G)
fig.show()

In [726]:
fig.write_image("../figures/authors_network_10tfidf.png")

In [728]:
textnet.draw_3d_network(G)

In [754]:
G_paul = textnet.construct_ego_network(G, "Paul of Tarsus", 10, reduced=True)

In [755]:
fig = textnet.draw_2d_network(G_paul)
fig.show()

In [756]:
fig.write_image("../figures/paul_egonet_10tfidf.png")

In [733]:
nx.to_pandas_adjacency(G_paul).astype(int)

,Paul of Tarsus,Clemens Romanus,Origenes,Theophilus Antiochenus,Eusebius,Epiphanius,Clement of Alexandria,Hermas,Athenagoras,Johnannine literature,Basil
Paul of Tarsus,0,2,2,3,2,5,0,0,0,0,0
Clemens Romanus,2,0,0,0,0,0,0,0,0,0,0
Origenes,2,0,0,0,0,0,0,0,0,3,0
Theophilus Antiochenus,3,0,0,0,0,0,0,0,0,0,2
Eusebius,2,0,0,0,0,0,0,0,0,0,0
Epiphanius,5,0,0,0,0,0,3,2,2,0,0
Clement of Alexandria,0,0,0,0,0,3,0,0,0,0,0
Hermas,0,0,0,0,0,2,0,0,0,0,0
Athenagoras,0,0,0,0,0,2,0,0,0,0,0
Johnannine literature,0,0,3,0,0,0,0,0,0,0,0


#  vectorization based on tfidf values

In [734]:
#tup_to_dict 
tfidfs_df_raw = pd.DataFrame([dict(tfidfs) for tfidfs in authors_df["dik_sents_TFIDF"].tolist()])
tfidfs_df_raw

,οὗτος,ἐντολή,πτωχός,διώκω,θεάομαι,ἀδελφός,ἐπιτρέπω,προεπεκαλέσασθέ,πληρόω,ἐκφέρω,...,ὑπερβάλλω,προειρημένου,παρακολουθέω,Ἰωσήφ,ἐγγίων,τελευτάω,ἀπεργάζομαι,τέσσαρες,ἡττηθέντα,κᾀκεῖνον
0,0.18466,0.18466,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.00714,NaN,NaN,0.03494,0.01016,NaN,NaN,NaN,0.01269,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.03078,NaN,NaN,0.02575,0.02113,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.04409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.04574,0.03691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,0.07851,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,0.01886,NaN,NaN,NaN,NaN,NaN,NaN,0.02423,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.01381,0.01541,0.03197,NaN,NaN,NaN,NaN,NaN,0.01578,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [757]:
selected_columns = []
for column in tfidfs_df_raw.columns:
    if len(tfidfs_df_raw[tfidfs_df_raw[column].notnull()]) > 1:
        selected_columns.append(column)
len(selected_columns)

861

In [758]:
tfidfs_df_filtered = tfidfs_df_raw[selected_columns]
tfidfs_df_filtered.fillna(0, inplace=True)

In [759]:
tfidfs_df_filtered

,οὗτος,ἐντολή,πτωχός,διώκω,θεάομαι,ἀδελφός,πληρόω,ἐκφέρω,ὅπλον,πρῶτος,...,τύραννος,τιμωρία,θεοφιλής,ὁμολογήσαντα,θελήσαντα,δουλεία,πατριάρχης,δύο,Μαρία,ἐγείρω
0,0.18466,0.18466,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,0.12457,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.00714,0.00000,0.00000,0.03494,0.01016,0.00000,0.01269,0.00000,0.04660,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.03078,0.00000,0.00000,0.02575,0.02113,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.04409,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5,0.04574,0.03691,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.01386,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
6,0.00000,0.00000,0.00000,0.07851,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
7,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
8,0.00000,0.01886,0.00000,0.00000,0.00000,0.00000,0.02423,0.00000,0.00000,0.01361,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
9,0.01381,0.01541,0.03197,0.00000,0.00000,0.00000,0.01578,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


# Tfidfs with cosine similarity

In [760]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD 
from sklearn.metrics.pairwise import cosine_similarity

In [761]:
tfidfs_sim_mat = cosine_similarity(tfidfs_df_filtered.to_numpy()).round(5)
tfidfs_dist_mat = 1 - cosine_similarity(tfidfs_df_filtered.to_numpy())
tfidfs_dist_mat = tfidfs_dist_mat.round(5)

In [762]:
authors = authors_df["author"].tolist()

In [763]:
tfidfs_sim_mat_df = pd.DataFrame(tfidfs_sim_mat, columns=authors, index=authors)
tfidfs_sim_mat_df

,Polycarp,Paul of Tarsus,Clemens Romanus,Pseudo-Justinus Martyr,Barnabas,Hermas,Pauline literature,Didache,GNT rest,Justinus Martyr,...,Luke (the evangelist),Ignatius Antiochenus,Basil,Maximus of Tyre,Hippolytus,Irenaeus,Tatianus,Martyrium Polycarpi,Mark,Dionysius Corinthius
Polycarp,1.00000,0.22181,0.23814,0.00707,0.07773,0.22399,0.12430,0.03068,0.17802,0.13459,...,0.09773,0.13008,0.04870,0.04291,0.02438,0.04281,0.0,0.0,0.0,0.0
Paul of Tarsus,0.22181,1.00000,0.36451,0.20356,0.18825,0.25099,0.21144,0.03965,0.35972,0.27068,...,0.18380,0.02245,0.20670,0.08929,0.08067,0.04672,0.0,0.0,0.0,0.0
Clemens Romanus,0.23814,0.36451,1.00000,0.08265,0.20547,0.31172,0.29054,0.06315,0.29542,0.30911,...,0.24766,0.00000,0.19421,0.05100,0.07233,0.10855,0.0,0.0,0.0,0.0
Pseudo-Justinus Martyr,0.00707,0.20356,0.08265,1.00000,0.05827,0.13644,0.08101,0.00000,0.16769,0.13493,...,0.03768,0.00000,0.12313,0.09990,0.03972,0.05339,0.0,0.0,0.0,0.0
Barnabas,0.07773,0.18825,0.20547,0.05827,1.00000,0.19986,0.15717,0.37385,0.18561,0.52461,...,0.16731,0.00000,0.08794,0.05815,0.01787,0.06708,0.0,0.0,0.0,0.0
Hermas,0.22399,0.25099,0.31172,0.13644,0.19986,1.00000,0.25905,0.12739,0.27988,0.24091,...,0.22410,0.00000,0.19954,0.08903,0.09794,0.13496,0.0,0.0,0.0,0.0
Pauline literature,0.12430,0.21144,0.29054,0.08101,0.15717,0.25905,1.00000,0.11154,0.24275,0.23870,...,0.15668,0.00000,0.18082,0.02560,0.04015,0.02968,0.0,0.0,0.0,0.0
Didache,0.03068,0.03965,0.06315,0.00000,0.37385,0.12739,0.11154,1.00000,0.02730,0.11109,...,0.08341,0.00000,0.05273,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0
GNT rest,0.17802,0.35972,0.29542,0.16769,0.18561,0.27988,0.24275,0.02730,1.00000,0.32517,...,0.22292,0.05269,0.15571,0.03767,0.04363,0.03382,0.0,0.0,0.0,0.0
Justinus Martyr,0.13459,0.27068,0.30911,0.13493,0.52461,0.24091,0.23870,0.11109,0.32517,1.00000,...,0.25626,0.02827,0.23503,0.09629,0.09710,0.08615,0.0,0.0,0.0,0.0


In [ ]:
set_with_dataframe(ECCE_AGT_overview.add_worksheet("tfidfs_sim", 1, 1), tfidfs_sim_mat_df, include_index=True)

In [764]:
G = nx.from_numpy_matrix(tfidfs_sim_mat)

authors_dict = dict(zip(range(len(authors)), authors))
G = nx.relabel_nodes(G, authors_dict)

total_weight = sum([int(n) for n in nx.get_edge_attributes(G, "weight").values()])
for (u, v) in G.edges:
    G[u][v]["norm_weight"] = round((G[u][v]["weight"] / total_weight), 5)
    G[u][v]["distance"] = round(1 / (G[u][v]["weight"]), 5)
    G[u][v]["norm_distance"] = round(1 / (G[u][v]["norm_weight"] ), 5)

edges_to_remove = []
for edge in G.edges:
    if edge[0] == edge[1]:
        edges_to_remove.append(edge)
G.remove_edges_from(edges_to_remove)

In [765]:
sim_threshold = 0.3
edges_to_remove = []
for edge in G.edges:
    if G[edge[0]][edge[1]]["weight"] < sim_threshold:
        edges_to_remove.append(edge)
G.remove_edges_from(edges_to_remove)

G.remove_nodes_from(list(nx.isolates(G)))

In [766]:
G.edges.data()

EdgeDataView([('Paul of Tarsus', 'Clemens Romanus', {'weight': 0.36451, 'norm_weight': 0.01402, 'distance': 2.74341, 'norm_distance': 71.32668}), ('Paul of Tarsus', 'GNT rest', {'weight': 0.35972, 'norm_weight': 0.01384, 'distance': 2.77994, 'norm_distance': 72.25434}), ('Paul of Tarsus', 'Clement of Alexandria', {'weight': 0.374, 'norm_weight': 0.01438, 'distance': 2.6738, 'norm_distance': 69.54103}), ('Paul of Tarsus', 'Origenes', {'weight': 0.46308, 'norm_weight': 0.01781, 'distance': 2.15945, 'norm_distance': 56.14823}), ('Paul of Tarsus', 'Theophilus Antiochenus', {'weight': 0.32512, 'norm_weight': 0.0125, 'distance': 3.07579, 'norm_distance': 80.0}), ('Paul of Tarsus', 'Eusebius', {'weight': 0.39914, 'norm_weight': 0.01535, 'distance': 2.50539, 'norm_distance': 65.14658}), ('Paul of Tarsus', 'Epiphanius', {'weight': 0.51618, 'norm_weight': 0.01985, 'distance': 1.93731, 'norm_distance': 50.37783}), ('Clemens Romanus', 'Hermas', {'weight': 0.31172, 'norm_weight': 0.01199, 'distance

In [768]:
fig = textnet.draw_2d_network(G)
fig.show()

In [745]:
fig = textnet.draw_2d_network(G)
fig.show()

In [746]:
fig.write_image("../figures/authors_network_cos-sim-tfidf.png")

In [747]:
G_paul = textnet.construct_ego_network(G, "Paul of Tarsus", 10, reduced=True)
fig = textnet.draw_2d_network(G_paul)
fig.show()

In [749]:
fig.write_image("../figures/paul_egonet_cos-sim-tfidf.png")